# SQL Script Collection

## Overview

This notebook contains custom SQL scripts and tools, as well as scripts which are collected from external sources and shared for everyone. This notebook hopefully would make it easy to find useful script during support activities. All are encouraged to share their on the job SQL scripts for diagnostic, troubleshooting and performance tuning or anything that is deemed helpful to others. Feel free to contribute and share using this notebook. 

## Tips

-   Make sure to check for existing script with similar result returned to avoid duplication of the same script or functionality.
-   You can use the standard CTRL + F to find keyword.
-   If there is similar script that is doing similar function, whenever possible please try to update and enhance the existing script first.
-   Avoid using lengthy and complex script, the simpler the better, as long as it does the job.
-   If you do have to share a very useful but complex script, perhaps it is better to create a new sub-folder in this Teams channel files tab and create another Jupyter (SQL) Notebook for it. For example: Implementing Automatic Sliding Window Partitioning.
-   To keep this notebook size small, before saving the file, please don't forget to clear all results (click on the **Clear Results** icon on the top right corner). 

Thank you!

## Scripts Collection

This section contains all scripts collected/contributed.

### Check SQL Services

This script checks for installed SQL services, their status (eg. started, stopped), service account used and basic cluster information.

In [ ]:
USE master
GO
SELECT servicename, startup_type_desc, status_desc, 
last_startup_time, service_account, is_clustered, cluster_nodename
FROM sys.dm_server_services OPTION (RECOMPILE);
GO

### SQL OS Version and Server Properties
Returns the version and important server properties.


In [ ]:
USE master;
GO
SELECT @@SERVERNAME AS [Server Name], @@VERSION AS [SQL Server and OS Version Info], SERVERPROPERTY('MachineName') AS [MachineName], SERVERPROPERTY('ServerName') AS [ServerName],  
SERVERPROPERTY('InstanceName') AS [Instance], SERVERPROPERTY('IsClustered') AS [IsClustered], 
SERVERPROPERTY('ComputerNamePhysicalNetBIOS') AS [ComputerNamePhysicalNetBIOS], 
SERVERPROPERTY('Edition') AS [Edition], SERVERPROPERTY('ProductLevel') AS [ProductLevel], 
SERVERPROPERTY('ProductVersion') AS [ProductVersion], SERVERPROPERTY('ProcessID') AS [ProcessID],
SERVERPROPERTY('Collation') AS [Collation], SERVERPROPERTY('IsFullTextInstalled') AS [IsFullTextInstalled], 
SERVERPROPERTY('IsIntegratedSecurityOnly') AS [IsIntegratedSecurityOnly],
SERVERPROPERTY('IsHadrEnabled') AS [IsHadrEnabled], SERVERPROPERTY('HadrManagerStatus') AS [HadrManagerStatus];
GO


### SQL Server Install Date

Returns when was SQL Server installed.

In [ ]:
USE master
GO
SELECT @@SERVERNAME AS [Server Name], createdate AS [SQL Server Install Date]  
FROM sys.syslogins 
WHERE [sid] = 0x010100000000000512000000;
GO

### SQL Server Authentication Mode

Returns information on SQL authentication mode (Windows or Mixed - Windows and SQL).

In [ ]:
USE master;
GO
SELECT @@SERVERNAME, CASE SERVERPROPERTY('IsIntegratedSecurityOnly')   
WHEN 1 THEN 'Windows Authentication'   
WHEN 0 THEN 'Windows and SQL Server Authentication'   
END as [Authentication Mode];
GO

### Windows Information
Gives you major OS version, service pack level, edition and language info for the OS.

In [ ]:
USE master
GO
SELECT windows_release, windows_service_pack_level, 
       windows_sku, os_language_version
FROM sys.dm_os_windows_info WITH (NOLOCK) OPTION (RECOMPILE);
GO

### SQL Server NUMA Node information
Gives you some useful information about the composition and relative load on your NUMA nodes.

In [ ]:
USE master
GO
SELECT node_id, node_state_desc, memory_node_id, online_scheduler_count, 
       active_worker_count, avg_load_balance 
FROM sys.dm_os_nodes WITH (NOLOCK) 
WHERE node_state_desc <> N'ONLINE DAC' OPTION (RECOMPILE);
GO

### System/Hardware Information
Gives you some good basic hardware information about your database server.

In [ ]:
USE master
GO
SELECT cpu_count AS [Logical CPU Count], hyperthread_ratio AS [Hyperthread Ratio],
cpu_count/hyperthread_ratio AS [Physical CPU Count], 
physical_memory_kb/1024 AS [Physical Memory (MB)], committed_target_kb/1024 AS [Committed Target Memory (MB)],
max_workers_count AS [Max Workers Count], affinity_type_desc AS [Affinity Type], 
sqlserver_start_time AS [SQL Server Start Time], virtual_machine_type_desc AS [Virtual Machine Type]
FROM sys.dm_os_sys_info WITH (NOLOCK) OPTION (RECOMPILE);
GO

### Available Memory


In [ ]:
SET QUOTED_IDENTIFIER ON
declare @ts bigint;
SELECT @ts = ms_ticks FROM sys.dm_os_sys_info (NOLOCK);

PRINT '-- AvailableRAM'
SELECT 
	Dateadd(ms, -1 * (@ts - t1.TIMESTAMP), Getdate()) as TimeStamp,
	t1.record.value('(./Record/ResourceMonitor/Notification)[1]', 'varchar(255)') as Notification,
	t1.record.value('(./Record/MemoryRecord/TotalPhysicalMemory)[1]', 'bigint')/1024 as TotalRAMInMB, 
	t1.record.value('(./Record/MemoryRecord/AvailablePhysicalMemory)[1]', 'bigint')/1024 as AvailableRAMInMb,
	t1.record.value('(./Record/MemoryRecord/AvailableVirtualAddressSpace)[1]', 'bigint')/1024 AS [Avail_VAS_MB],
	t1.record.value('(./Record/MemoryRecord/TotalPageFile)[1]', 'bigint')/1024 AS [Total_Pagefile_MB],
	t1.record.value('(./Record/MemoryRecord/AvailablePageFile)[1]', 'bigint')/1024 AS [Avail_Pagefile_MB]
FROM (SELECT MAX([TIMESTAMP]) AS [TIMESTAMP], CONVERT(xml, record) AS record 
	FROM sys.dm_os_ring_buffers (NOLOCK)
	WHERE ring_buffer_type = N'RING_BUFFER_RESOURCE_MONITOR'
		--AND record LIKE '%RESOURCE_MEMPHYSICAL%'
	GROUP BY record) AS t1

Union All

SELECT 
	Getdate(),system_memory_state_desc,total_physical_memory_kb/1024, available_physical_memory_kb/1024, 0,total_page_file_kb / 1014,available_page_file_kb /1024
FROM sys.dm_os_sys_memory
order by 
	TIMESTAMP

### Get Cluster Information
You will see no results if your instance is not clustered.

In [ ]:
USE master
GO
SELECT VerboseLogging, SqlDumperDumpFlags, SqlDumperDumpPath, 
       SqlDumperDumpTimeOut, FailureConditionLevel, HealthCheckTimeout
FROM sys.dm_os_cluster_properties WITH (NOLOCK) OPTION (RECOMPILE);
GO

### Cluster Nodes Information
Knowing which node owns the cluster resources is critical. Especially when you are installing Windows or SQL Server updates. You will see no results if your instance is not clustered.

In [ ]:
USE master
GO
SELECT NodeName, status_description, is_current_owner
FROM sys.dm_os_cluster_nodes WITH (NOLOCK) OPTION (RECOMPILE);
GO

### Server/Instance Configuration
Returns all instance configuration settings (alternative of sp_configure).

In [ ]:
USE master;
GO
SELECT * FROM sys.configurations;
GO

### TCP End Point Listerner Information
Helpful for network and connectivity troubleshooting.

In [ ]:
USE master;
GO
SELECT listener_id, ip_address, is_ipv4, port, type_desc, state_desc, start_time
FROM sys.dm_tcp_listener_states WITH (NOLOCK) OPTION (RECOMPILE);
GO

### SQL Server Registry
This lets you safely read some SQL Server related information from the Windows Registry.



In [ ]:
USE master;
GO
SELECT registry_key, value_name, value_data
FROM sys.dm_server_registry WITH (NOLOCK) OPTION (RECOMPILE);
GO

### Checks For Any SQL Memory Dump
This will not return any rows if you have not had any memory dumps (which is a good thing).

In [ ]:
USE master
GO
SELECT [filename], creation_time, size_in_bytes
FROM sys.dm_server_memory_dumps WITH (NOLOCK) OPTION (RECOMPILE);
GO

### Show Enabled Trace Flags (Global)

In [ ]:
DBCC TRACESTATUS(-1)

### Backup History

In [ ]:
PRINT '-- BackupHistory'

SELECT 
    databasename = bs.database_name,
    filegroupname = bf.filegroup_name,
    pagesize = bf.page_size,
    filenumber = bf.file_number,
    backeduppagecount = bf.backed_up_page_count,
    filetype = bf.file_type,
    sourcefileblocksize = bf.source_file_block_size,
    filesize = bf.file_size,
    logicalname = bf.logical_name,
    physicaldrive = bf.physical_drive,
    physicalname = bf.physical_name,
    state = bf.[state],
    statedesc = bf.state_desc,
    BackupFilebackupsize = bf.backup_size,
    BackupFileisreadonly = bf.is_readonly,
    ispresent = bf.is_present,
    position = bs.[position],
    expirationdate = bs.expiration_date,
    softwarevendorid = bs.software_vendor_id,
    name = bs.name,
    username = bs.user_name,
    databasecreationdate = bs.database_creation_date,
    backupstartdate = bs.backup_start_date,
    backupfinishdate = bs.backup_finish_date,
    type = bs.type,
    compatibilitylevel = bs.compatibility_level,
    databaseversion = bs.database_version,
    BackupSetbackupsize = bs.backup_size,
    servername = bs.server_name,
    machinename = bs.machine_name,
    collationname = bs.collation_name,
    ispasswordprotected = bs.is_password_protected,
    recoverymodel = bs.recovery_model,
    hasbulkloggeddata = bs.has_bulk_logged_data,
    issnapshot = bs.is_snapshot,
    BackupSetisreadonly = bs.is_readonly,
    issingleuser = bs.is_single_user,
    hasbackupchecksums = bs.has_backup_checksums,
    isdamaged = bs.is_damaged,
    beginslogchain = bs.begins_log_chain,
    isforceoffline = bs.is_force_offline,
    iscopyonly = bs.is_copy_only,
    compressedbackupsize = bs.compressed_backup_size,
    mirror = bm.mirror, 
	physical_device_name

FROM
    msdb..backupfile bf
    JOIN msdb..backupset bs
    ON bf.backup_set_id = bs.backup_set_id
    JOIN msdb..backupmediafamily bm
    ON bs.media_set_id = bm.media_set_id
ORDER BY
    backup_finish_date DESC

### Query Optimizer Information

In [ ]:
PRINT '-- OptimizerInfo'
SELECT  *
FROM    sys.dm_exec_query_optimizer_info

### Resource Governor

In [ ]:
select * from sys.dm_resource_governor_configuration
select * from sys.dm_resource_governor_resource_pools
select * from sys.dm_resource_governor_workload_groups



### Ring Buffer Exception Monitor


In [ ]:
SET NOCOUNT ON
SET ANSI_NULLS ON
SET QUOTED_IDENTIFIER ON
PRINT '-- RingBufferExceptionMonitor'
SELECT 
	'RING_BUFFER_EXCEPTION' AS RingType, 
	DATEADD(ms, -1 * ((SELECT ms_ticks FROM sys.dm_os_sys_info) - [timestamp]), GETDATE()) AS EventTime,
	*
FROM (
	SELECT 
		record.value('(./Record/@id)[1]', 'bigint') AS record_id,
		record.value('(./Record/Exception/Task/@address)[1]', 'varchar(200)') AS TaskAddress,
		record.value('(./Record/Exception/Error)[1]', 'int') AS ErrorNumber,
		record.value('(./Record/Exception/Severity)[1]', 'int') AS Severity,
		record.value('(./Record/Exception/State)[1]', 'int') AS State,
		record.value('(./Record/Exception/UserDefined)[1]', 'int') AS UserDefined,
		timestamp
	FROM (
		SELECT timestamp, CONVERT(XML, record) AS record 
		FROM sys.dm_os_ring_buffers 
		WHERE ring_buffer_type = N'RING_BUFFER_EXCEPTION'
		) AS x
	) AS y
	JOIN sys.messages m ON y.ErrorNumber = m.message_id AND m.severity = y.severity AND language_id = 1033
ORDER BY record_id DESC

### Ring Buffer Resource Monitor


In [ ]:
SELECT 
	'RING_BUFFER_RESOURCE_MONITOR' AS RingType, 
	DATEADD(ms, -1 * ((SELECT ms_ticks FROM sys.dm_os_sys_info) - [timestamp]), GETDATE()) AS EventTime,
	*
FROM (
	SELECT 
		record.value('(./Record/@id)[1]', 'bigint') AS record_id,
		record.value('(./Record/ResourceMonitor/Notification)[1]', 'varchar(100)') AS Notification,
		record.value('(./Record/ResourceMonitor/IndicatorsProcess)[1]', 'bit') AS IndicatorsProcess,
		record.value('(./Record/ResourceMonitor/IndicatorsSystem)[1]', 'bit') AS IndicatorsSystem,
		record.value('(./Record/ResourceMonitor/NodeId)[1]', 'bigint') AS NodeId,
		record.value('(./Record/ResourceMonitor/Effect/@type)[1]', 'varchar(100)') AS EffectType1,
		record.value('(./Record/ResourceMonitor/Effect/@state)[1]', 'varchar(100)') AS EffectState1,
		record.value('(./Record/ResourceMonitor/Effect/@reversed)[1]', 'varchar(100)') AS EffectReversed1,
		record.value('(./Record/ResourceMonitor/Effect/@type)[2]', 'varchar(100)') AS EffectType2,
		record.value('(./Record/ResourceMonitor/Effect/@state)[2]', 'varchar(100)') AS EffectState2,
		record.value('(./Record/ResourceMonitor/Effect/@reversed)[2]', 'varchar(100)') AS EffectReversed2,
		record.value('(./Record/ResourceMonitor/Effect/@type)[3]', 'varchar(100)') AS EffectType3,
		record.value('(./Record/ResourceMonitor/Effect/@state)[3]', 'varchar(100)') AS EffectState3,
		record.value('(./Record/ResourceMonitor/Effect/@reversed)[3]', 'varchar(100)') AS EffectReversed3,
		record.value('(./Record/MemoryNode/@id)[1]', 'bigint') AS PageFaults,
		record.value('(./Record/MemoryNode/ReservedMemory)[1]', 'bigint') AS ReservedMemory,
		record.value('(./Record/MemoryNode/CommittedMemory)[1]', 'bigint') AS CommittedMemory,
		record.value('(./Record/MemoryNode/SharedMemory)[1]', 'bigint') AS SharedMemory,
		record.value('(./Record/MemoryNode/AWEMemory)[1]', 'bigint') AS AWEMemory,
		record.value('(./Record/MemoryNode/SinglePagesMemory)[1]', 'bigint') AS SinglePagesMemory,
		record.value('(./Record/MemoryNode/MultiplePagesMemory)[1]', 'bigint') AS MultiplePagesMemory,
		record.value('(./Record/MemoryRecord/MemoryUtilization)[1]', 'bigint') AS MemoryUtilization,
		record.value('(./Record/MemoryRecord/TotalPhysicalMemory)[1]', 'bigint') AS TotalPhysicalMemory,
		record.value('(./Record/MemoryRecord/AvailablePhysicalMemory)[1]', 'bigint') AS AvailablePhysicalMemory,
		record.value('(./Record/MemoryRecord/TotalPageFile)[1]', 'bigint') AS TotalPageFile,
		record.value('(./Record/MemoryRecord/AvailablePageFile)[1]', 'bigint') AS AvailablePageFile,
		record.value('(./Record/MemoryRecord/TotalVirtualAddressSpace)[1]', 'bigint') AS TotalVirtualAddressSpace,
		record.value('(./Record/MemoryRecord/AvailableVirtualAddressSpace)[1]', 'bigint') AS AvailableVirtualAddressSpace,
		record.value('(./Record/MemoryRecord/AvailableExtendedVirtualAddressSpace)[1]', 'bigint') AS AvailableExtendedVirtualAddressSpace,
		timestamp
	FROM (
		SELECT timestamp, CONVERT(XML, record) AS record 
		FROM sys.dm_os_ring_buffers 
		WHERE ring_buffer_type = N'RING_BUFFER_RESOURCE_MONITOR'
		) AS x
	) AS y
ORDER BY record_id DESC


### Ring Buffer Scheduler Monitor


In [ ]:
SET NOCOUNT ON
SET ANSI_NULLS ON
SET QUOTED_IDENTIFIER ON
PRINT '-- RingBufferSchedulerMonitor'
SELECT
	'RING_BUFFER_SCHEDULER_MONITOR' AS RingType, 
	DATEADD(ms, -1 * ((SELECT ms_ticks FROM sys.dm_os_sys_info) - [timestamp]), GETDATE()) AS EventTime,
	*
FROM (
	SELECT 
		record.value('(./Record/@id)[1]', 'bigint') AS record_id,
		record.value('(./Record/SchedulerMonitorEvent/SystemHealth/SystemIdle)[1]', 'bigint') AS SystemIdle,
		record.value('(./Record/SchedulerMonitorEvent/SystemHealth/ProcessUtilization)[1]', 'bigint') AS SQLProcessUtilization,
		record.value('(./Record/SchedulerMonitorEvent/SystemHealth/UserModeTime)[1]', 'bigint') AS UserModeTime,
		record.value('(./Record/SchedulerMonitorEvent/SystemHealth/KernelModeTime)[1]', 'bigint') AS KernelModeTime,
		record.value('(./Record/SchedulerMonitorEvent/SystemHealth/PageFaults)[1]', 'bigint') AS PageFaults,
		record.value('(./Record/SchedulerMonitorEvent/SystemHealth/WorkingSetDelta)[1]', 'bigint') AS WorkingSetDelta,
		record.value('(./Record/SchedulerMonitorEvent/SystemHealth/MemoryUtilization)[1]', 'bigint') AS MemoryUtilPct,
		timestamp
	FROM (
		SELECT timestamp, CONVERT(XML, record) AS record 
		FROM sys.dm_os_ring_buffers 
		WHERE ring_buffer_type = N'RING_BUFFER_SCHEDULER_MONITOR'
		AND record LIKE '%<SystemHealth>%') AS x
	) AS y
ORDER BY record_id DESC

### Suspect Pages


In [ ]:
select d.name, p.* 
from msdb..suspect_pages p
LEFT JOIN sys.databases d on p.database_id = p.page_id

### Databases file information

Things to look at:

-   Are data files and log files on different drives?
-   Is everything on the C: drive?
-   Is TempDB on dedicated drives?
-   Is there only one TempDB data file?
-   Are all of the TempDB data files the same size?
-   Are there multiple data files for user databases?

In [ ]:
USE master;
GO
SELECT DB_NAME([database_id])AS [Database Name], 
       [file_id], name, physical_name, type_desc, state_desc, 
       CONVERT( bigint, size/128.0) AS [Total Size in MB]
FROM sys.master_files WITH (NOLOCK)
WHERE [database_id] <> 32767
ORDER BY DB_NAME([database_id]) OPTION (RECOMPILE);
GO

### Databases Storage Space Information
Shows you the free space on the LUNs where you have database data or log files.

In [ ]:
USE master;
GO
SELECT DB_NAME(f.database_id) AS [DatabaseName], f.file_id, 
vs.volume_mount_point, vs.total_bytes, vs.available_bytes, 
CAST(CAST(vs.available_bytes AS FLOAT)/ CAST(vs.total_bytes AS FLOAT) AS DECIMAL(18,3)) * 100 AS [Space Free %]
FROM sys.master_files AS f
CROSS APPLY sys.dm_os_volume_stats(f.database_id, f.file_id) AS vs
ORDER BY f.database_id OPTION (RECOMPILE);
GO

### Database Recovery Model, Log Reuse Information and Compatibility Level
Things to look at:
- How many databases are on the instance?
- What recovery models are they using?
- What is the log reuse wait description?
- How full are the transaction logs ?
- What compatibility level are they on?
- What is the Page Verify Option?
- Make sure auto_shrink and auto_close are not enabled!


In [ ]:
USE master;
GO
SELECT db.[name] AS [Database Name], db.recovery_model_desc AS [Recovery Model], 
db.log_reuse_wait_desc AS [Log Reuse Wait Description], 
ls.cntr_value AS [Log Size (KB)], lu.cntr_value AS [Log Used (KB)],
CAST(CAST(lu.cntr_value AS FLOAT) / CAST(ls.cntr_value AS FLOAT)AS DECIMAL(18,2)) * 100 AS [Log Used %], 
db.[compatibility_level] AS [DB Compatibility Level], 
db.page_verify_option_desc AS [Page Verify Option], db.is_auto_create_stats_on, db.is_auto_update_stats_on,
db.is_auto_update_stats_async_on, db.is_parameterization_forced, 
db.snapshot_isolation_state_desc, db.is_read_committed_snapshot_on,
db.is_auto_close_on, db.is_auto_shrink_on, db.target_recovery_time_in_seconds
FROM sys.databases AS db
INNER JOIN sys.dm_os_performance_counters AS lu 
ON db.name = lu.instance_name
INNER JOIN sys.dm_os_performance_counters AS ls 
ON db.name = ls.instance_name
WHERE lu.counter_name LIKE N'Log File(s) Used Size (KB)%' 
AND ls.counter_name LIKE N'Log File(s) Size (KB)%'
AND ls.cntr_value > 0 OPTION (RECOMPILE);
GO

### Database Files Average IO Stall
Helps determine which database files on the entire instance have the most I/O bottlenecks. This can help you decide whether certain LUNs are overloaded and whether you might want to move some files to a different location.


In [ ]:
USE master;
GO
SELECT DB_NAME(fs.database_id) AS [Database Name], mf.physical_name, io_stall_read_ms, num_of_reads,
CAST(io_stall_read_ms/(1.0 + num_of_reads) AS NUMERIC(10,1)) AS [avg_read_stall_ms],io_stall_write_ms, 
num_of_writes,CAST(io_stall_write_ms/(1.0+num_of_writes) AS NUMERIC(10,1)) AS [avg_write_stall_ms],
io_stall_read_ms + io_stall_write_ms AS [io_stalls], num_of_reads + num_of_writes AS [total_io],
CAST((io_stall_read_ms + io_stall_write_ms)/(1.0 + num_of_reads + num_of_writes) AS NUMERIC(10,1)) 
AS [avg_io_stall_ms]
FROM sys.dm_io_virtual_file_stats(null,null) AS fs
INNER JOIN sys.master_files AS mf WITH (NOLOCK)
ON fs.database_id = mf.database_id
AND fs.[file_id] = mf.[file_id]
ORDER BY avg_io_stall_ms DESC OPTION (RECOMPILE);
GO

### Database CPU Stats
Helps determine which database is using the most CPU resources on the instance.

In [ ]:
USE master;
GO
WITH DB_CPU_Stats
AS
(SELECT DatabaseID, DB_Name(DatabaseID) AS [DatabaseName], SUM(total_worker_time) AS [CPU_Time_Ms]
 FROM sys.dm_exec_query_stats AS qs
 CROSS APPLY (SELECT CONVERT(int, value) AS [DatabaseID] 
              FROM sys.dm_exec_plan_attributes(qs.plan_handle)
              WHERE attribute = N'dbid') AS F_DB
 GROUP BY DatabaseID)
SELECT ROW_NUMBER() OVER(ORDER BY [CPU_Time_Ms] DESC) AS [row_num],
       DatabaseName, [CPU_Time_Ms], 
       CAST([CPU_Time_Ms] * 1.0 / SUM([CPU_Time_Ms]) OVER() * 100.0 AS DECIMAL(5, 2)) AS [CPUPercent]
FROM DB_CPU_Stats
WHERE DatabaseID > 4 -- system databases
AND DatabaseID <> 32767 -- ResourceDB
ORDER BY row_num OPTION (RECOMPILE);
GO


### Database Memory Stats
Helps determine which database is using the most memory resources on the instance.

In [ ]:
USE master;
GO
WITH DB_Memory_Usage AS (
SELECT CASE database_id WHEN 32767 THEN 'ResourceDB' ELSE DB_NAME(database_id) END as [DB_Name],
	   CAST(COUNT(*)* 8/1024 AS DECIMAL(12,2)) as [Cached_MB]
FROM sys.dm_os_buffer_descriptors
GROUP BY db_name(database_id) ,database_id)
SELECT ROW_NUMBER() OVER(ORDER BY [Cached_MB] DESC) AS [row_num], [DB_Name], [Cached_MB], CAST([Cached_MB]/SUM([Cached_MB]) OVER() * 100.0 AS DECIMAL(5,2)) AS [Percent]
FROM DB_Memory_Usage
ORDER BY [Cached_MB] DESC;
GO

### Wait Statistics
[SQL Server Wait Stats - Tell me where it hurts...](https://www.sqlskills.com/blogs/paul/wait-statistics-or-please-tell-me-where-it-hurts/)

In [ ]:
USE master;
GO
-- Last updated October 14, 2020
WITH [Waits] AS
    (SELECT
        [wait_type],
        [wait_time_ms] / 1000.0 AS [WaitS],
        ([wait_time_ms] - [signal_wait_time_ms]) / 1000.0 AS [ResourceS],
        [signal_wait_time_ms] / 1000.0 AS [SignalS],
        [waiting_tasks_count] AS [WaitCount],
        100.0 * [wait_time_ms] / SUM ([wait_time_ms]) OVER() AS [Percentage],
        ROW_NUMBER() OVER(ORDER BY [wait_time_ms] DESC) AS [RowNum]
    FROM sys.dm_os_wait_stats
    WHERE [wait_type] NOT IN (
        -- These wait types are almost 100% never a problem and so they are
        -- filtered out to avoid them skewing the results. Click on the URL
        -- for more information.
        N'BROKER_EVENTHANDLER', -- https://www.sqlskills.com/help/waits/BROKER_EVENTHANDLER
        N'BROKER_RECEIVE_WAITFOR', -- https://www.sqlskills.com/help/waits/BROKER_RECEIVE_WAITFOR
        N'BROKER_TASK_STOP', -- https://www.sqlskills.com/help/waits/BROKER_TASK_STOP
        N'BROKER_TO_FLUSH', -- https://www.sqlskills.com/help/waits/BROKER_TO_FLUSH
        N'BROKER_TRANSMITTER', -- https://www.sqlskills.com/help/waits/BROKER_TRANSMITTER
        N'CHECKPOINT_QUEUE', -- https://www.sqlskills.com/help/waits/CHECKPOINT_QUEUE
        N'CHKPT', -- https://www.sqlskills.com/help/waits/CHKPT
        N'CLR_AUTO_EVENT', -- https://www.sqlskills.com/help/waits/CLR_AUTO_EVENT
        N'CLR_MANUAL_EVENT', -- https://www.sqlskills.com/help/waits/CLR_MANUAL_EVENT
        N'CLR_SEMAPHORE', -- https://www.sqlskills.com/help/waits/CLR_SEMAPHORE
 
        -- Maybe comment this out if you have parallelism issues
        N'CXCONSUMER', -- https://www.sqlskills.com/help/waits/CXCONSUMER
 
        -- Maybe comment these four out if you have mirroring issues
        N'DBMIRROR_DBM_EVENT', -- https://www.sqlskills.com/help/waits/DBMIRROR_DBM_EVENT
        N'DBMIRROR_EVENTS_QUEUE', -- https://www.sqlskills.com/help/waits/DBMIRROR_EVENTS_QUEUE
        N'DBMIRROR_WORKER_QUEUE', -- https://www.sqlskills.com/help/waits/DBMIRROR_WORKER_QUEUE
        N'DBMIRRORING_CMD', -- https://www.sqlskills.com/help/waits/DBMIRRORING_CMD
        N'DIRTY_PAGE_POLL', -- https://www.sqlskills.com/help/waits/DIRTY_PAGE_POLL
        N'DISPATCHER_QUEUE_SEMAPHORE', -- https://www.sqlskills.com/help/waits/DISPATCHER_QUEUE_SEMAPHORE
        N'EXECSYNC', -- https://www.sqlskills.com/help/waits/EXECSYNC
        N'FSAGENT', -- https://www.sqlskills.com/help/waits/FSAGENT
        N'FT_IFTS_SCHEDULER_IDLE_WAIT', -- https://www.sqlskills.com/help/waits/FT_IFTS_SCHEDULER_IDLE_WAIT
        N'FT_IFTSHC_MUTEX', -- https://www.sqlskills.com/help/waits/FT_IFTSHC_MUTEX
  
       -- Maybe comment these six out if you have AG issues
        N'HADR_CLUSAPI_CALL', -- https://www.sqlskills.com/help/waits/HADR_CLUSAPI_CALL
        N'HADR_FILESTREAM_IOMGR_IOCOMPLETION', -- https://www.sqlskills.com/help/waits/HADR_FILESTREAM_IOMGR_IOCOMPLETION
        N'HADR_LOGCAPTURE_WAIT', -- https://www.sqlskills.com/help/waits/HADR_LOGCAPTURE_WAIT
        N'HADR_NOTIFICATION_DEQUEUE', -- https://www.sqlskills.com/help/waits/HADR_NOTIFICATION_DEQUEUE
        N'HADR_TIMER_TASK', -- https://www.sqlskills.com/help/waits/HADR_TIMER_TASK
        N'HADR_WORK_QUEUE', -- https://www.sqlskills.com/help/waits/HADR_WORK_QUEUE
 
        N'KSOURCE_WAKEUP', -- https://www.sqlskills.com/help/waits/KSOURCE_WAKEUP
        N'LAZYWRITER_SLEEP', -- https://www.sqlskills.com/help/waits/LAZYWRITER_SLEEP
        N'LOGMGR_QUEUE', -- https://www.sqlskills.com/help/waits/LOGMGR_QUEUE
        N'MEMORY_ALLOCATION_EXT', -- https://www.sqlskills.com/help/waits/MEMORY_ALLOCATION_EXT
        N'ONDEMAND_TASK_QUEUE', -- https://www.sqlskills.com/help/waits/ONDEMAND_TASK_QUEUE
        N'PARALLEL_REDO_DRAIN_WORKER', -- https://www.sqlskills.com/help/waits/PARALLEL_REDO_DRAIN_WORKER
        N'PARALLEL_REDO_LOG_CACHE', -- https://www.sqlskills.com/help/waits/PARALLEL_REDO_LOG_CACHE
        N'PARALLEL_REDO_TRAN_LIST', -- https://www.sqlskills.com/help/waits/PARALLEL_REDO_TRAN_LIST
        N'PARALLEL_REDO_WORKER_SYNC', -- https://www.sqlskills.com/help/waits/PARALLEL_REDO_WORKER_SYNC
        N'PARALLEL_REDO_WORKER_WAIT_WORK', -- https://www.sqlskills.com/help/waits/PARALLEL_REDO_WORKER_WAIT_WORK
        N'PREEMPTIVE_OS_FLUSHFILEBUFFERS', -- https://www.sqlskills.com/help/waits/PREEMPTIVE_OS_FLUSHFILEBUFFERS
        N'PREEMPTIVE_XE_GETTARGETSTATE', -- https://www.sqlskills.com/help/waits/PREEMPTIVE_XE_GETTARGETSTATE
        N'PWAIT_ALL_COMPONENTS_INITIALIZED', -- https://www.sqlskills.com/help/waits/PWAIT_ALL_COMPONENTS_INITIALIZED
        N'PWAIT_DIRECTLOGCONSUMER_GETNEXT', -- https://www.sqlskills.com/help/waits/PWAIT_DIRECTLOGCONSUMER_GETNEXT
        N'QDS_PERSIST_TASK_MAIN_LOOP_SLEEP', -- https://www.sqlskills.com/help/waits/QDS_PERSIST_TASK_MAIN_LOOP_SLEEP
        N'QDS_ASYNC_QUEUE', -- https://www.sqlskills.com/help/waits/QDS_ASYNC_QUEUE
        N'QDS_CLEANUP_STALE_QUERIES_TASK_MAIN_LOOP_SLEEP',
            -- https://www.sqlskills.com/help/waits/QDS_CLEANUP_STALE_QUERIES_TASK_MAIN_LOOP_SLEEP
        N'QDS_SHUTDOWN_QUEUE', -- https://www.sqlskills.com/help/waits/QDS_SHUTDOWN_QUEUE
        N'REDO_THREAD_PENDING_WORK', -- https://www.sqlskills.com/help/waits/REDO_THREAD_PENDING_WORK
        N'REQUEST_FOR_DEADLOCK_SEARCH', -- https://www.sqlskills.com/help/waits/REQUEST_FOR_DEADLOCK_SEARCH
        N'RESOURCE_QUEUE', -- https://www.sqlskills.com/help/waits/RESOURCE_QUEUE
        N'SERVER_IDLE_CHECK', -- https://www.sqlskills.com/help/waits/SERVER_IDLE_CHECK
        N'SLEEP_BPOOL_FLUSH', -- https://www.sqlskills.com/help/waits/SLEEP_BPOOL_FLUSH
        N'SLEEP_DBSTARTUP', -- https://www.sqlskills.com/help/waits/SLEEP_DBSTARTUP
        N'SLEEP_DCOMSTARTUP', -- https://www.sqlskills.com/help/waits/SLEEP_DCOMSTARTUP
        N'SLEEP_MASTERDBREADY', -- https://www.sqlskills.com/help/waits/SLEEP_MASTERDBREADY
        N'SLEEP_MASTERMDREADY', -- https://www.sqlskills.com/help/waits/SLEEP_MASTERMDREADY
        N'SLEEP_MASTERUPGRADED', -- https://www.sqlskills.com/help/waits/SLEEP_MASTERUPGRADED
        N'SLEEP_MSDBSTARTUP', -- https://www.sqlskills.com/help/waits/SLEEP_MSDBSTARTUP
        N'SLEEP_SYSTEMTASK', -- https://www.sqlskills.com/help/waits/SLEEP_SYSTEMTASK
        N'SLEEP_TASK', -- https://www.sqlskills.com/help/waits/SLEEP_TASK
        N'SLEEP_TEMPDBSTARTUP', -- https://www.sqlskills.com/help/waits/SLEEP_TEMPDBSTARTUP
        N'SNI_HTTP_ACCEPT', -- https://www.sqlskills.com/help/waits/SNI_HTTP_ACCEPT
        N'SOS_WORK_DISPATCHER', -- https://www.sqlskills.com/help/waits/SOS_WORK_DISPATCHER
        N'SP_SERVER_DIAGNOSTICS_SLEEP', -- https://www.sqlskills.com/help/waits/SP_SERVER_DIAGNOSTICS_SLEEP
        N'SQLTRACE_BUFFER_FLUSH', -- https://www.sqlskills.com/help/waits/SQLTRACE_BUFFER_FLUSH
        N'SQLTRACE_INCREMENTAL_FLUSH_SLEEP', -- https://www.sqlskills.com/help/waits/SQLTRACE_INCREMENTAL_FLUSH_SLEEP
        N'SQLTRACE_WAIT_ENTRIES', -- https://www.sqlskills.com/help/waits/SQLTRACE_WAIT_ENTRIES
        N'VDI_CLIENT_OTHER', -- https://www.sqlskills.com/help/waits/VDI_CLIENT_OTHER
        N'WAIT_FOR_RESULTS', -- https://www.sqlskills.com/help/waits/WAIT_FOR_RESULTS
        N'WAITFOR', -- https://www.sqlskills.com/help/waits/WAITFOR
        N'WAITFOR_TASKSHUTDOWN', -- https://www.sqlskills.com/help/waits/WAITFOR_TASKSHUTDOWN
        N'WAIT_XTP_RECOVERY', -- https://www.sqlskills.com/help/waits/WAIT_XTP_RECOVERY
        N'WAIT_XTP_HOST_WAIT', -- https://www.sqlskills.com/help/waits/WAIT_XTP_HOST_WAIT
        N'WAIT_XTP_OFFLINE_CKPT_NEW_LOG', -- https://www.sqlskills.com/help/waits/WAIT_XTP_OFFLINE_CKPT_NEW_LOG
        N'WAIT_XTP_CKPT_CLOSE', -- https://www.sqlskills.com/help/waits/WAIT_XTP_CKPT_CLOSE
        N'XE_DISPATCHER_JOIN', -- https://www.sqlskills.com/help/waits/XE_DISPATCHER_JOIN
        N'XE_DISPATCHER_WAIT', -- https://www.sqlskills.com/help/waits/XE_DISPATCHER_WAIT
        N'XE_TIMER_EVENT' -- https://www.sqlskills.com/help/waits/XE_TIMER_EVENT
        )
    AND [waiting_tasks_count] > 0
    )
SELECT
    MAX ([W1].[wait_type]) AS [WaitType],
    CAST (MAX ([W1].[WaitS]) AS DECIMAL (16,2)) AS [Wait_S],
    CAST (MAX ([W1].[ResourceS]) AS DECIMAL (16,2)) AS [Resource_S],
    CAST (MAX ([W1].[SignalS]) AS DECIMAL (16,2)) AS [Signal_S],
    MAX ([W1].[WaitCount]) AS [WaitCount],
    CAST (MAX ([W1].[Percentage]) AS DECIMAL (5,2)) AS [Percentage],
    CAST ((MAX ([W1].[WaitS]) / MAX ([W1].[WaitCount])) AS DECIMAL (16,4)) AS [AvgWait_S],
    CAST ((MAX ([W1].[ResourceS]) / MAX ([W1].[WaitCount])) AS DECIMAL (16,4)) AS [AvgRes_S],
    CAST ((MAX ([W1].[SignalS]) / MAX ([W1].[WaitCount])) AS DECIMAL (16,4)) AS [AvgSig_S],
    CAST ('https://www.sqlskills.com/help/waits/' + MAX ([W1].[wait_type]) as XML) AS [Help/Info URL]
FROM [Waits] AS [W1]
INNER JOIN [Waits] AS [W2] ON [W2].[RowNum] <= [W1].[RowNum]
GROUP BY [W1].[RowNum]
HAVING SUM ([W2].[Percentage]) - MAX( [W1].[Percentage] ) < 95; -- percentage threshold
GO

### Get Logins and The Sessions They Have
This can help characterize your workload and determine whether you are seeing a normal level of activity.

In [ ]:
USE master;
GO
SELECT login_name, COUNT(session_id) AS [session_count] 
FROM sys.dm_exec_sessions WITH (NOLOCK)
GROUP BY login_name
ORDER BY COUNT(session_id) DESC OPTION (RECOMPILE);
GO

### Get Average Task Counts (run multiple times)
Sustained values above 10 suggest further investigation in that area
- High Avg Task Counts are often caused by blocking or other resource contention
- High Avg Runnable Task Counts are a good sign of CPU pressure
- High Avg Pending DiskIO Counts are a sign of disk pressure


In [ ]:
USE master;
GO
SELECT AVG(current_tasks_count) AS [Avg Task Count], 
AVG(runnable_tasks_count) AS [Avg Runnable Task Count],
AVG(pending_disk_io_count) AS [Avg Pending DiskIO Count]
FROM sys.dm_os_schedulers WITH (NOLOCK)
WHERE scheduler_id < 255 OPTION (RECOMPILE);
GO

### SQL Server Process Address Space Info 
Shows whether locked pages is enabled, among other things:
- You want to see 0 for process_physical_memory_low
- You want to see 0 for process_virtual_memory_low
This indicates that you are not under internal memory pressure.


In [ ]:
USE master;
GO
SELECT physical_memory_in_use_kb,locked_page_allocations_kb, 
       page_fault_count, memory_utilization_percentage, 
       available_commit_limit_kb, process_physical_memory_low, 
       process_virtual_memory_low
FROM sys.dm_os_process_memory WITH (NOLOCK) OPTION (RECOMPILE);
GO

### SQL Server Performance Counters

Returns common performance counter's values from sys.dm\_os\_performance\_counters.

In [ ]:
USE master;
GO
SELECT object_name, counter_name, instance_name, cntr_value 
FROM sys.dm_os_performance_counters
WHERE counter_name LIKE 'Buffer cache hit ratio  %'
OR counter_name LIKE 'Page life expectancy%'
OR counter_name LIKE 'LAzy writes/sec%'
OR counter_name LIKE 'Page life expectancy%'
OR counter_name LIKE 'Temp Tables%'
OR counter_name LIKE 'Logins/sec'
OR counter_name LIKE 'Connection Reset%'
OR counter_name LIKE 'Logouts/sec'
OR counter_name LIKE 'User Connections%'
OR counter_name LIKE 'Transactions/sec%'
OR counter_name LIKE 'Active Transactions%'
OR counter_name LIKE 'Mars Deadlocks%'
OR counter_name LIKE 'Processes Blocked%'
OR counter_name LIKE 'Number of Deadlocks/sec%'
OR counter_name LIKE 'Lock Timeouts (timeout > 0)/sec%'
OR counter_name LIKE 'Batch Requests/sec%'
OR counter_name LIKE '%active cursor%'
OR counter_name LIKE 'Memory Grants%'
OR counter_name LIKE '%Server Memory%'
AND (instance_name LIKE '_Total%')
ORDER BY counter_name ASC;
GO

### FInd Single-Used AdHoc Query Plans That Are Bloating The Plan Cache
Gives you the text and size of single-use ad-hoc queries that waste space in the plan cache
Enabling 'optimize for ad hoc workloads' for the instance can help (SQL Server 2008 and above only)
Enabling forced parameterization for the database can help, but test first!

In [ ]:
USE master;
GO
SELECT TOP(20) [text] AS [QueryText], cp.size_in_bytes
FROM sys.dm_exec_cached_plans AS cp WITH (NOLOCK)
CROSS APPLY sys.dm_exec_sql_text(plan_handle) 
WHERE cp.cacheobjtype = N'Compiled Plan' 
AND cp.objtype = N'Adhoc' 
AND cp.usecounts = 1
ORDER BY cp.size_in_bytes DESC OPTION (RECOMPILE);
GO

### Latches With Most Waiting Requests


In [ ]:
SELECT TOP 15 * FROM sys.dm_os_latch_stats
ORDER BY waiting_requests_count DESC

### Find Sleeping Sessions with Open Transaction


In [ ]:
  select datediff(minute, s.last_request_end_time, getdate()) as minutes_asleep,
         s.session_id,
         db_name(s.database_id) as database_name,
         s.host_name,
         s.host_process_id,
         t.text as last_sql,
         s.program_name,
		 s.open_transaction_count
    from sys.dm_exec_connections c
    join sys.dm_exec_sessions s
         on c.session_id = s.session_id
   outer apply sys.dm_exec_sql_text(c.most_recent_sql_handle) t
   where s.is_user_process = 1 and s.open_transaction_count >= 1
         and s.status = 'sleeping'
         and datediff(second, s.last_request_end_time, getdate()) > 60
   order by s.last_request_end_time;

### Duplicate Indexes

In [ ]:
;with IndexColumns AS(
select distinct  schema_name (o.schema_id) as 'SchemaName',object_name(o.object_id) as TableName, i.Name as IndexName, o.object_id,i.index_id,i.type,
(select case key_ordinal when 0 then NULL else '['+col_name(k.object_id,column_id) +'] ' + CASE WHEN is_descending_key=1 THEN 'Desc' ELSE 'Asc' END end as [data()]
from sys.index_columns  (NOLOCK) as k
where k.object_id = i.object_id
and k.index_id = i.index_id
order by key_ordinal, column_id
for xml path('')) as cols,
case when i.index_id=1 then 
(select '['+name+']' as [data()]
from sys.columns  (NOLOCK) as c
where c.object_id = i.object_id
and c.column_id not in (select column_id from sys.index_columns  (NOLOCK) as kk    where kk.object_id = i.object_id and kk.index_id = i.index_id)
order by column_id
for xml path(''))
else (select '['+col_name(k.object_id,column_id) +']' as [data()]
from sys.index_columns  (NOLOCK) as k
where k.object_id = i.object_id
and k.index_id = i.index_id and is_included_column=1 and k.column_id not in (Select column_id from sys.index_columns kk where k.object_id=kk.object_id and kk.index_id=1)
order by key_ordinal, column_id
for xml path('')) end as inc
from sys.indexes  (NOLOCK) as i
inner join sys.objects o  (NOLOCK) on i.object_id =o.object_id 
inner join sys.index_columns ic  (NOLOCK) on ic.object_id =i.object_id and ic.index_id =i.index_id
inner join sys.columns c  (NOLOCK) on c.object_id = ic.object_id and c.column_id = ic.column_id
where  o.type = 'U' and i.index_id <>0 and i.type <>3 and i.type <>5 and i.type <>6 and i.type <>7 
group by o.schema_id,o.object_id,i.object_id,i.Name,i.index_id,i.type
),
DuplicatesTable AS
(SELECT    ic1.SchemaName,ic1.TableName,ic1.IndexName,ic1.object_id, ic2.IndexName as DuplicateIndexName, 
CASE WHEN ic1.index_id=1 THEN ic1.cols + ' (Clustered)' WHEN ic1.inc = '' THEN ic1.cols  WHEN ic1.inc is NULL THEN ic1.cols ELSE ic1.cols + ' INCLUDE ' + ic1.inc END as IndexCols, 
ic1.index_id
from IndexColumns ic1 join IndexColumns ic2 on ic1.object_id = ic2.object_id
and ic1.index_id < ic2.index_id and ic1.cols = ic2.cols
and (ISNULL(ic1.inc,'') = ISNULL(ic2.inc,'')  OR ic1.index_id=1 )
)
SELECT SchemaName,TableName, IndexName,DuplicateIndexName, IndexCols, index_id, object_id, 0 AS IsXML
FROM DuplicatesTable dt
ORDER BY 1,2,3


### Redundant Indexes


In [ ]:
;with IndexColumns AS(
select distinct  schema_name (o.schema_id) as 'SchemaName',object_name(o.object_id) as TableName, i.Name as IndexName, o.object_id,i.index_id,i.type,
(select case key_ordinal when 0 then NULL else '['+col_name(k.object_id,column_id) +']' end as [data()]
from sys.index_columns  (NOLOCK) as k
where k.object_id = i.object_id
and k.index_id = i.index_id
order by key_ordinal, column_id
for xml path('')) as cols,
(select case key_ordinal when 0 then NULL else '['+col_name(k.object_id,column_id) +'] ' + CASE WHEN is_descending_key=1 THEN 'Desc' ELSE 'Asc' END end as [data()]
from sys.index_columns  (NOLOCK) as k
where k.object_id = i.object_id
and k.index_id = i.index_id
order by key_ordinal, column_id
for xml path('')) as colsWithSortOrder,
case when i.index_id=1 then 
(select '['+name+']' as [data()]
from sys.columns  (NOLOCK) as c
where c.object_id = i.object_id
and c.column_id not in (select column_id from sys.index_columns  (NOLOCK) as kk    where kk.object_id = i.object_id and kk.index_id = i.index_id)
order by column_id for xml path(''))
else
(select '['+col_name(k.object_id,column_id) +']' as [data()]
from sys.index_columns  (NOLOCK) as k
where k.object_id = i.object_id
and k.index_id = i.index_id and is_included_column=1 and k.column_id not in (Select column_id from sys.index_columns kk where k.object_id=kk.object_id and kk.index_id=1)
order by key_ordinal, column_id for xml path('')) end as inc
from sys.indexes  (NOLOCK) as i
inner join sys.objects o  (NOLOCK) on i.object_id =o.object_id 
inner join sys.index_columns ic  (NOLOCK) on ic.object_id =i.object_id and ic.index_id =i.index_id
inner join sys.columns c  (NOLOCK) on c.object_id = ic.object_id and c.column_id = ic.column_id
where  o.type = 'U' and i.index_id <>0 and i.type <>3 and i.type <>5 and i.type <>6 and i.type <>7
group by o.schema_id,o.object_id,i.object_id,i.Name,i.index_id,i.type
), ResultTable AS
(SELECT    ic1.SchemaName,ic1.TableName,ic1.IndexName,ic1.object_id, ic2.IndexName as RedundantIndexName, CASE WHEN ic1.index_id=1 THEN ic1.colsWithSortOrder + ' (Clustered)' WHEN ic1.inc = '' THEN ic1.colsWithSortOrder  WHEN ic1.inc is NULL THEN ic1.colsWithSortOrder ELSE ic1.colsWithSortOrder + ' INCLUDE ' + ic1.inc END as IndexCols, 
CASE WHEN ic2.index_id=1 THEN ic2.colsWithSortOrder + ' (Clustered)' WHEN ic2.inc = '' THEN ic2.colsWithSortOrder  WHEN ic2.inc is NULL THEN ic2.colsWithSortOrder ELSE ic2.colsWithSortOrder + ' INCLUDE ' + ic2.inc END as RedundantIndexCols, ic1.index_id
,ic1.cols col1,ic2.cols col2
from IndexColumns ic1 join IndexColumns ic2 on ic1.object_id = ic2.object_id
and ic1.index_id <> ic2.index_id and not (ic1.colsWithSortOrder = ic2.colsWithSortOrder and ISNULL(ic1.inc,'') = ISNULL(ic2.inc,''))
and not (ic1.index_id=1 AND ic1.cols = ic2.cols ) and ic1.cols like REPLACE (ic2.cols , '[','[[]') + '%'
)
SELECT SchemaName,TableName, IndexName, IndexCols, RedundantIndexName, RedundantIndexCols, object_id, index_id
FROM ResultTable
ORDER BY 1,2,3,5


### Find FK without Index

In [ ]:
 /* Foreign keys */
;WITH FKTable 
as(
    SELECT schema_name(o.schema_id) AS 'parent_schema_name',object_name(FKC.parent_object_id) 'parent_table_name',
    object_name(constraint_object_id) AS 'constraint_name',schema_name(RO.Schema_id) AS 'referenced_schema',object_name(referenced_object_id) AS 'referenced_table_name',
    (SELECT '['+col_name(k.parent_object_id,parent_column_id) +']' AS [data()]
      FROM sys.foreign_key_columns (NOLOCK) AS k
      INNER JOIN sys.foreign_keys (NOLOCK)
      ON k.constraint_object_id =object_id
      AND k.constraint_object_id =FKC.constraint_object_id
      ORDER BY constraint_column_id
      FOR XML PATH('') 
    ) AS 'parent_colums',
    (SELECT '['+col_name(k.referenced_object_id,referenced_column_id) +']' AS [data()]
      FROM sys.foreign_key_columns (NOLOCK) AS k
      INNER JOIN sys.foreign_keys (NOLOCK)
      ON k.constraint_object_id =object_id
      AND k.constraint_object_id =FKC.constraint_object_id
      ORDER BY constraint_column_id
      FOR XML PATH('') 
    ) AS 'referenced_columns'
  FROM sys.foreign_key_columns FKC (NOLOCK)
  INNER JOIN sys.objects o (NOLOCK) ON FKC.parent_object_id = o.object_id
  INNER JOIN sys.objects RO (NOLOCK) ON FKC.referenced_object_id = RO.object_id
  WHERE o.object_id in (SELECT object_id FROM sys.objects (NOLOCK) WHERE type ='U') AND RO.object_id in (SELECT object_id FROM sys.objects (NOLOCK) WHERE type ='U')
  group by o.schema_id,RO.schema_id,FKC.parent_object_id,constraint_object_id,referenced_object_id
),
/* Index Columns */
IndexColumnsTable AS
(
  SELECT distinct schema_name (o.schema_id) AS 'schema_name',object_name(o.object_id) AS TableName,
  (SELECT case key_ordinal when 0 then NULL else '['+col_name(k.object_id,column_id) +']' end AS [data()]
    FROM sys.index_columns (NOLOCK) AS k
    WHERE k.object_id = i.object_id
    AND k.index_id = i.index_id
    ORDER BY key_ordinal, column_id
    FOR XML PATH('')
  ) AS cols
  FROM sys.indexes (NOLOCK) AS i
  INNER JOIN sys.objects o (NOLOCK) ON i.object_id =o.object_id 
  INNER JOIN sys.index_columns ic (NOLOCK) ON ic.object_id =i.object_id AND ic.index_id =i.index_id
  INNER JOIN sys.columns c (NOLOCK) ON c.object_id = ic.object_id AND c.column_id = ic.column_id
  WHERE i.object_id in (SELECT object_id FROM sys.objects (NOLOCK) WHERE type ='U') AND i.index_id > 0
  group by o.schema_id,o.object_id,i.object_id,i.Name,i.index_id,i.type
)
SELECT 
  fk.parent_schema_name AS SchemaName,
  fk.parent_table_name AS TableName,
  fk.constraint_name AS ConstraintName,
  fk.referenced_schema AS ReferencedSchemaName,
  fk.referenced_table_name AS ReferencedTableName
FROM FKTable fk 
WHERE (SELECT COUNT(*) AS NbIndexes  FROM IndexColumnsTable ict  WHERE fk.parent_schema_name = ict.schema_name AND fk.parent_table_name = ict.TableName      AND fk.parent_colums = ict.cols
  ) = 0
 




### Tables without Clustered Index (Heaps)


In [ ]:
SELECT DISTINCT 
schema_name(so.schema_id) AS 'SchemaName',object_name(so.object_id) AS 'TableName',so.object_id AS 'object_id', max(dmv.rows) AS 'ApproximateRows', 
CASE objectproperty(MAX(so.object_id), 'TableHasClustIndex') WHEN 0 THEN count(si.index_id) - 1 ELSE COUNT(si.index_id) END as 'IndexCount', MAX(d.ColumnCount) AS 'ColumnCount'
FROM sys.objects so (NOLOCK)
JOIN sys.indexes si (NOLOCK) ON so.object_id = si.object_id AND so.type in (N'U',N'V') 
JOIN sysindexes dmv (NOLOCK) ON so.object_id = dmv.id AND si.index_id = dmv.indid
FULL OUTER JOIN (SELECT object_id, count(1) AS ColumnCount FROM sys.columns (NOLOCK) GROUP BY object_id) d 
ON d.object_id = so.object_id
WHERE so.is_ms_shipped = 0
AND so.object_id NOT IN (SELECT major_id FROM sys.extended_properties (NOLOCK) WHERE name = N'microsoft_database_tools_support')
AND indexproperty(so.object_id, si.name, 'IsStatistics') = 0
GROUP BY so.schema_id, so.object_id
HAVING (objectproperty(max(so.object_id), 'TableHasClustIndex') = 0 
--AND COUNT(si.index_id)-1 > 0
)
ORDER BY SchemaName, TableName;


### Tables without Any Index

In [ ]:
SELECT DISTINCT 
schema_name(so.schema_id) AS 'SchemaName',object_name(so.object_id) AS 'TableName',so.object_id AS 'object_id', max(dmv.rows) AS 'ApproximateRows', 
CASE objectproperty(MAX(so.object_id), 'TableHasClustIndex') WHEN 0 THEN count(si.index_id) - 1 ELSE COUNT(si.index_id) END as 'IndexCount', MAX(d.ColumnCount) AS 'ColumnCount'
FROM sys.objects so (NOLOCK)
JOIN sys.indexes si (NOLOCK) ON so.object_id = si.object_id AND so.type in (N'U',N'V') 
JOIN sysindexes dmv (NOLOCK) ON so.object_id = dmv.id AND si.index_id = dmv.indid
FULL OUTER JOIN (SELECT object_id, count(1) AS ColumnCount FROM sys.columns (NOLOCK) GROUP BY object_id) d 
ON d.object_id = so.object_id
WHERE so.is_ms_shipped = 0
AND so.object_id NOT IN (SELECT major_id FROM sys.extended_properties (NOLOCK) WHERE name = N'microsoft_database_tools_support')
AND indexproperty(so.object_id, si.name, 'IsStatistics') = 0
GROUP BY so.schema_id, so.object_id
HAVING (objectproperty(max(so.object_id), 'TableHasClustIndex') = 0 
--AND COUNT(si.index_id)-1 > 0
)
ORDER BY SchemaName, TableName;

### Missing Indexes


In [ ]:
SELECT mig.index_group_handle,mid.index_handle, migs.avg_total_user_cost as AvgTotalUserCostThatCouldbeReduced, migs.avg_user_impact as AvgPercentageBenefit,
'CREATE INDEX missing_index_' + CONVERT (varchar, mig.index_group_handle) + '_' + CONVERT (varchar, mid.index_handle)
  + ' ON ' + mid.statement 
  + ' (' + ISNULL (mid.equality_columns,'')
    + CASE WHEN mid.equality_columns IS NOT NULL AND mid.inequality_columns IS NOT NULL THEN ',' ELSE '' END
    + ISNULL (mid.inequality_columns, '')
  + ')'
  + ISNULL (' INCLUDE (' + mid.included_columns + ')', '') AS create_index_statement 
FROM sys.dm_db_missing_index_groups mig
INNER JOIN sys.dm_db_missing_index_group_stats migs ON migs.group_handle = mig.index_group_handle
INNER JOIN sys.dm_db_missing_index_details mid ON mig.index_handle = mid.index_handle
ORDER BY migs.avg_user_impact desc, migs.avg_total_user_cost desc

### Sessions Transaction Isolation Level

In [ ]:
SELECT DISTINCT DB_NAME(database_id) as database_name, program_name, host_name,client_interface_name, login_name, status, open_transaction_count, CASE transaction_isolation_level 
WHEN 0 THEN 'Unspecified' 
WHEN 1 THEN 'ReadUncommitted' 
WHEN 2 THEN 'ReadCommitted' 
WHEN 3 THEN 'Repeatable' 
WHEN 4 THEN 'Serializable' 
WHEN 5 THEN 'Snapshot' END AS transaction_isolation_level 
FROM sys.dm_exec_sessions 
WHERE is_user_process = 1 AND database_id > 4;


### Index Usage Stats - Scan, Seek, Lookup


In [ ]:
SELECT OBJECT_NAME(IX.OBJECT_ID) Table_Name
	   ,IX.name AS Index_Name
	   ,IX.type_desc Index_Type
	   ,SUM(PS.[used_page_count]) * 8 IndexSizeKB
	   ,IXUS.user_seeks AS NumOfSeeks
	   ,IXUS.user_scans AS NumOfScans
	   ,IXUS.user_lookups AS NumOfLookups
	   ,IXUS.user_updates AS NumOfUpdates
	   ,IXUS.last_user_seek AS LastSeek
	   ,IXUS.last_user_scan AS LastScan
	   ,IXUS.last_user_lookup AS LastLookup
	   ,IXUS.last_user_update AS LastUpdate
FROM sys.indexes IX
INNER JOIN sys.dm_db_index_usage_stats IXUS ON IXUS.index_id = IX.index_id AND IXUS.OBJECT_ID = IX.OBJECT_ID
INNER JOIN sys.dm_db_partition_stats PS on PS.object_id=IX.object_id
WHERE OBJECTPROPERTY(IX.OBJECT_ID,'IsUserTable') = 1
GROUP BY OBJECT_NAME(IX.OBJECT_ID) ,IX.name ,IX.type_desc ,IXUS.user_seeks ,IXUS.user_scans ,IXUS.user_lookups,IXUS.user_updates ,IXUS.last_user_seek ,IXUS.last_user_scan ,IXUS.last_user_lookup ,IXUS.last_user_update

### Index Usage Stats - Update, Insert, Delete


In [ ]:
SELECT OBJECT_NAME(IXOS.OBJECT_ID)  Table_Name 
       ,IX.name  Index_Name
	   ,IX.type_desc Index_Type
	   ,SUM(PS.[used_page_count]) * 8 IndexSizeKB
       ,IXOS.LEAF_INSERT_COUNT NumOfInserts
       ,IXOS.LEAF_UPDATE_COUNT NumOfupdates
       ,IXOS.LEAF_DELETE_COUNT NumOfDeletes
FROM   SYS.DM_DB_INDEX_OPERATIONAL_STATS (NULL,NULL,NULL,NULL ) IXOS 
INNER JOIN SYS.INDEXES AS IX ON IX.OBJECT_ID = IXOS.OBJECT_ID AND IX.INDEX_ID =    IXOS.INDEX_ID 
	INNER JOIN sys.dm_db_partition_stats PS on PS.object_id=IX.object_id
WHERE  OBJECTPROPERTY(IX.[OBJECT_ID],'IsUserTable') = 1
GROUP BY OBJECT_NAME(IXOS.OBJECT_ID), IX.name, IX.type_desc,IXOS.LEAF_INSERT_COUNT, IXOS.LEAF_UPDATE_COUNT,IXOS.LEAF_DELETE_COUNT

### Find SQL Triggers

In [ ]:
DECLARE @QRY VARCHAR(MAX) ='';

SELECT @QRY =@QRY + ' select  ' + QUOTENAME(name,'''') + ' COLLATE SQL_Latin1_General_CP1_CI_AS AS database_name 
    ,OBJECT_NAME(TR.parent_id) COLLATE SQL_Latin1_General_CP1_CI_AS AS table_name 
	,name COLLATE SQL_Latin1_General_CP1_CI_AS AS trigger_name
	,parent_class_desc
	,type_desc
	,create_date
	,is_disabled
FROM ['+name+'].SYS.TRIGGERS TR
UNION ALL
' 
FROM SYS.DATABASES
WHERE name not IN ('master', 'model', 'msdb', 'tempdb', 'resource',
       'distribution' , 'reportserver', 'reportservertempdb','jiradb')

SELECT @QRY = SUBSTRING(@QRY,1,LEN(@QRY)-12)

EXEC( @QRY)

### Top 10 Most Executed Stored Procedures

In [ ]:
SELECT  TOP 10 DB_NAME(database_id) AS DatabaseName,
        OBJECT_NAME(object_id,database_id) AS ProcName,
		execution_count,
        total_elapsed_time / 1000000 TotElapsed,
		(total_elapsed_time/1000000)/execution_count AvgElapsed,
        last_elapsed_time / 1000000 LastElapsed,
        min_elapsed_time / 1000000 MinElapsed,
        max_elapsed_time / 1000000 MaxElapsed,
        max_logical_reads / 1000000 MaxRead,
        max_logical_writes / 1000000 MaxWrite
FROM    sys.dm_exec_procedure_stats
WHERE database_id > = 5 and database_id <= 32766
ORDER BY execution_count DESC


### Top 5 Most Duration (avg) Stored Procedures

In [ ]:
SELECT TOP 5 CASE WHEN a.database_id = 32767 then 'Resource' ELSE DB_NAME(database_id)END AS DBName
      ,OBJECT_SCHEMA_NAME(object_id,a.database_id) AS [SCHEMA_NAME]  
      ,OBJECT_NAME(object_id,a.database_id)AS [OBJECT_NAME]
      ,a.cached_time
      ,a.last_execution_time
      ,a.execution_count
	  ,qs.refcounts, qs.usecounts
      ,a.total_worker_time / a.execution_count AS AVG_CPU
      ,a.total_elapsed_time / a.execution_count AS AVG_ELAPSED
      ,a.total_logical_reads / a.execution_count AS AVG_LOGICAL_READS
      ,a.total_logical_writes / a.execution_count AS AVG_LOGICAL_WRITES
      ,a.total_physical_reads  / a.execution_count AS AVG_PHYSICAL_READS
	  ,qp.query_plan
FROM sys.dm_exec_procedure_stats a
JOIN [sys].[dm_exec_cached_plans] AS [qs] ON [a].[plan_handle] = [qs].[plan_handle]
CROSS APPLY [sys].[dm_exec_query_plan]([qs].[plan_handle]) AS [qp]
WHERE (a.database_id > 4 AND DB_NAME(database_id) <> 'Resource') 
AND qp.query_plan IS NOT NULL
ORDER BY AVG_ELAPSED DESC

### Top 5 Most Reads (avg) Stored Procedures 

In [ ]:
SELECT TOP 5 CASE WHEN a.database_id = 32767 then 'Resource' ELSE DB_NAME(database_id)END AS DBName
      ,OBJECT_SCHEMA_NAME(object_id,a.database_id) AS [SCHEMA_NAME]  
      ,OBJECT_NAME(object_id,a.database_id)AS [OBJECT_NAME]
      ,a.cached_time
      ,a.last_execution_time
      ,a.execution_count
	  ,qs.refcounts, qs.usecounts
      ,a.total_worker_time / a.execution_count AS AVG_CPU
      ,a.total_elapsed_time / a.execution_count AS AVG_ELAPSED
      ,a.total_logical_reads / a.execution_count AS AVG_LOGICAL_READS
      ,a.total_logical_writes / a.execution_count AS AVG_LOGICAL_WRITES
      ,a.total_physical_reads  / a.execution_count AS AVG_PHYSICAL_READS
	  ,qp.query_plan
FROM sys.dm_exec_procedure_stats a
JOIN [sys].[dm_exec_cached_plans] AS [qs] ON [a].[plan_handle] = [qs].[plan_handle]
CROSS APPLY [sys].[dm_exec_query_plan]([qs].[plan_handle]) AS [qp]
WHERE (a.database_id > 4 AND DB_NAME(database_id) <> 'Resource') 
AND qp.query_plan IS NOT NULL
ORDER BY AVG_LOGICAL_READS DESC

### Top 5 Most CPU Time (avg) Stored Procedures

In [ ]:
SELECT TOP 5 CASE WHEN a.database_id = 32767 then 'Resource' ELSE DB_NAME(database_id)END AS DBName
      ,OBJECT_SCHEMA_NAME(object_id,a.database_id) AS [SCHEMA_NAME]  
      ,OBJECT_NAME(object_id,a.database_id)AS [OBJECT_NAME]
      ,a.cached_time
      ,a.last_execution_time
      ,a.execution_count
	  ,qs.refcounts, qs.usecounts
      ,a.total_worker_time / a.execution_count AS AVG_CPU
      ,a.total_elapsed_time / a.execution_count AS AVG_ELAPSED
      ,a.total_logical_reads / a.execution_count AS AVG_LOGICAL_READS
      ,a.total_logical_writes / a.execution_count AS AVG_LOGICAL_WRITES
      ,a.total_physical_reads  / a.execution_count AS AVG_PHYSICAL_READS
	  ,qp.query_plan
FROM sys.dm_exec_procedure_stats a
JOIN [sys].[dm_exec_cached_plans] AS [qs] ON [a].[plan_handle] = [qs].[plan_handle]
CROSS APPLY [sys].[dm_exec_query_plan]([qs].[plan_handle]) AS [qp]
WHERE (a.database_id > 4 AND DB_NAME(database_id) <> 'Resource') 
AND qp.query_plan IS NOT NULL
ORDER BY AVG_CPU DESC

### Top 5 Expensive Queries


In [ ]:
SELECT TOP 10 SUBSTRING(qt.TEXT, (qs.statement_start_offset/2)+1,
((CASE qs.statement_end_offset
WHEN -1 THEN DATALENGTH(qt.TEXT)
ELSE qs.statement_end_offset
END - qs.statement_start_offset)/2)+1) QueryText,
qs.execution_count,
qs.total_logical_reads, qs.last_logical_reads,
qs.total_logical_writes, qs.last_logical_writes,
qs.total_worker_time,
qs.last_worker_time,
qs.total_elapsed_time/1000000 total_elapsed_time_in_S,
qs.last_elapsed_time/1000000 last_elapsed_time_in_S,
qs.last_execution_time,
qp.query_plan
FROM sys.dm_exec_query_stats qs
CROSS APPLY sys.dm_exec_sql_text(qs.sql_handle) qt
CROSS APPLY sys.dm_exec_query_plan(qs.plan_handle) qp
ORDER BY qs.total_logical_reads DESC -- logical reads
-- ORDER BY qs.total_logical_writes DESC -- logical writes
-- ORDER BY qs.total_worker_time DESC -- CPU time

### Find Queries That Use Parallelism


In [ ]:
;WITH XMLNAMESPACES
(DEFAULT 'http://schemas.microsoft.com/sqlserver/2004/07/showplan')
SELECT
    query_plan AS CompleteQueryPlan
    , n.value('(@StatementText)[1]', 'VARCHAR(4000)') AS StatementText
    , n.value('(@StatementOptmLevel)[1]', 'VARCHAR(25)') AS StatementOptimizationLevel
    , n.value('(@StatementSubTreeCost)[1]', 'VARCHAR(128)') AS StatementSubTreeCost
    , n.query('.') AS ParallelSubTreeXML
    , cp.usecounts
    , cp.size_in_bytes
FROM sys.dm_exec_cached_plans AS cp
CROSS APPLY sys.dm_exec_query_plan(plan_handle) AS qp
CROSS APPLY query_plan.nodes('/ShowPlanXML/BatchSequence/Batch/Statements/StmtSimple') AS qn(n)
WHERE n.query('.').exist('//RelOp[@PhysicalOp="Parallelism"]') = 1;

### Calculate Average Subtree Cost of All Queries That Use Parallelism
Helpful to determine the right value for Cost Threshold of Parallelism.

In [ ]:
CREATE TABLE #SubtreeCost(StatementSubtreeCost DECIMAL(18,2));
;WITH XMLNAMESPACES
(DEFAULT 'http://schemas.microsoft.com/sqlserver/2004/07/showplan')
INSERT INTO #SubtreeCost
SELECT
    CAST(n.value('(@StatementSubTreeCost)[1]', 'VARCHAR(128)') AS DECIMAL(18,2))
FROM sys.dm_exec_cached_plans AS cp
CROSS APPLY sys.dm_exec_query_plan(plan_handle) AS qp
CROSS APPLY query_plan.nodes('/ShowPlanXML/BatchSequence/Batch/Statements/StmtSimple') AS qn(n)
WHERE n.query('.').exist('//RelOp[@PhysicalOp="Parallelism"]') = 1;
SELECT AVG(StatementSubtreeCost) AS AverageSubtreeCost
FROM #SubtreeCost;
 
DROP TABLE #SubtreeCost;

### 

### 